In [1]:
#FP树的类定义
class treeNode:
    def __init__(self,nameValue,numOccur,parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None #链接相似的元素项
        self.parent = parentNode
        self.children = {}
    
    #对count增加给定值
    def inc(self,numOccur):
        self.count += numOccur
        
    #将树以文本形式显示
    def disp(self,ind=1):
        print (" "*ind, self.name, " ",self.count)
        for child in self.children.values():
            child.disp(ind+1)
            

In [2]:
rootNode = treeNode("pyramid",9,None)

In [3]:
rootNode.children['eye'] = treeNode('eye',13,None)

In [4]:
rootNode.disp()

  pyramid   9
   eye   13


In [60]:
#FP树构建函数
def createTree(dataset, minSup=1):
    headerTable={}
    for trans in dataset:
        for item in trans:
            headerTable[item] = headerTable.get(item,0) + dataset[trans]#项和出现次数
    for k in list(headerTable.keys()):#字典遍历过程中不能修改字典元素，将其改为list
        if headerTable[k] < minSup: #删除不满足最小支持度的元素项
            del(headerTable[k])
    freqItemSet = set(headerTable.keys())#频繁项集
    if len(freqItemSet) == 0: return None,None
    for k in headerTable:
        headerTable[k] = [headerTable[k],None]#None保存每种元素指向第一个元素项的指针，headerTable是头指针表
    #print (headerTable)
    retTree = treeNode('Null Set',1, None)
    for tranSet, count in dataset.items():
        localD = {}
        for item in tranSet:
            if item in freqItemSet:
                localD[item] = headerTable[item][0]#local[item]存储的是全局元素频率
                #print (localD[item])
        if len(localD) > 0: #根据全局元素频率对每个trans的元素排序
            #print (localD)
            #print (localD.items())
            orderedItems = [v[0] for v in sorted(localD.items(),key=lambda p :p[1],reverse=True)]
            #print (orderedItems)
            updateTree(orderedItems,retTree,headerTable,count)#对每个trans更新树，这里count是trans的重复次数
    return retTree,headerTable

def updateTree(items,inTree,headerTable,count):
    if items[0] in inTree.children:#测试trans中第一个元素是否作为子节点存在，如存在则更新计数
        inTree.children[items[0]].inc(count)
    else:
        inTree.children[items[0]] = treeNode(items[0],count,inTree)#如trans中第一个元素不存在，创建一个节点
        if headerTable[items[0]][1] == None:#头指针更新指向新节点
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
           
            updateHeader(headerTable[items[0]][1],inTree.children[items[0]])
    if len(items)>1:#不断调用自身，每次去掉首部的元素
        updateTree(items[1::],inTree.children[items[0]],headerTable,count)
        
def updateHeader(nodeToTest,targetNode):
    #print(nodeToTest,targetNode)
    #print (nodeToTest.nodeLink)
    while(nodeToTest.nodeLink != None):#头指针nodeLink指向一个子树
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode
    #print (nodeToTest.nodeLink)
    
    
        

In [61]:
dataset = [['r','z','h','j','p'],
          ['z','y','x','w','v','u','t','s'],
          ['z'],
          ['r','x','n','o','s'],
          ['y','r','x','z','q','t','p'],
          ['y','z','x','e','q','s','t','m']]

def createInitSet(dataset):
    retDict={}
    for trans in dataset:
        retDict[frozenset(trans)]=1
    return retDict


In [62]:
initSet = createInitSet(dataset)

In [63]:
initSet

{frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [64]:
myFPtree,myHeaderTab = createTree(initSet,3)

In [65]:
myFPtree.disp()

  Null Set   1
   z   5
    r   1
    x   3
     s   2
      y   2
       t   2
     r   1
      y   1
       t   1
   x   1
    r   1
     s   1


In [66]:
myHeaderTab

{'r': [3, <__main__.treeNode at 0x7f84995ab790>],
 'z': [5, <__main__.treeNode at 0x7f84995ab650>],
 'x': [4, <__main__.treeNode at 0x7f84995aba50>],
 's': [3, <__main__.treeNode at 0x7f84995ab8d0>],
 'y': [3, <__main__.treeNode at 0x7f84995ab850>],
 't': [3, <__main__.treeNode at 0x7f84995ab250>]}

In [67]:
#发现以给定元素项结尾的所有路径的函数
def ascendTree(leafNode,prefixPath):
    if leafNode.parent!=None:
        prefixPath.append(leafNode.name)#前缀路径是介于所查找元素项与树根节点之间的所有内容
        ascendTree(leafNode.parent,prefixPath)
        
def findPrefixPath(basePat,treeNode):#条件模式基condPats，是以所查找元素项为结尾的路径集合，每条路径都是一条前缀路径
    condPats={}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode,prefixPath)
        if len(prefixPath)>1:
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

        

In [68]:
findPrefixPath('x',myHeaderTab['x'][1])

{frozenset({'z'}): 3}

In [69]:
findPrefixPath('x',myHeaderTab['r'][1])

{frozenset({'z'}): 1, frozenset({'x'}): 1, frozenset({'x', 'z'}): 1}

In [72]:
#递归查找频繁项集的mineTree函数
def mineTree(inTree,headerTable,minSup,preFix,freqItemList):
    #print (sorted(headerTable.items(),key=lambda p:p[0]))
    bigL = [v[0] for v in sorted(headerTable.items(),key=lambda p:p[0])]
    for basePat in bigL:
        #print (basePat)
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        freqItemList.append(newFreqSet)
        condPattBases = findPrefixPath(basePat,headerTable[basePat][1])
        myCondTree,myHead = createTree(condPattBases,minSup)
        if myHead != None:
            mineTree(myCondTree,myHead,minSup,newFreqSet,freqItemList)
            print ('conditional tree for:',newFreqSet)
            
            myCondTree.disp(1)

In [73]:
freqItems = []
mineTree(myFPtree,myHeaderTab,3,set([]),freqItems)

conditional tree for: {'s'}
  Null Set   1
   x   3
conditional tree for: {'x', 't'}
  Null Set   1
   y   3
conditional tree for: {'x', 'z', 't'}
  Null Set   1
   y   3
conditional tree for: {'z', 't'}
  Null Set   1
   y   3
    x   3
conditional tree for: {'t'}
  Null Set   1
   y   3
    x   3
     z   3
conditional tree for: {'x'}
  Null Set   1
   z   3
conditional tree for: {'y', 'z'}
  Null Set   1
   x   3
conditional tree for: {'y'}
  Null Set   1
   x   3
    z   3


In [74]:
freqItems

[{'r'},
 {'s'},
 {'s', 'x'},
 {'t'},
 {'t', 'x'},
 {'t', 'x', 'y'},
 {'t', 'y'},
 {'t', 'z'},
 {'t', 'x', 'z'},
 {'t', 'x', 'y', 'z'},
 {'t', 'y', 'z'},
 {'x'},
 {'x', 'z'},
 {'y'},
 {'x', 'y'},
 {'y', 'z'},
 {'x', 'y', 'z'},
 {'z'}]